In [1]:
import pandas as pd

In [2]:
#list of numbers from 1 to 100
df_numbers=pd.read_csv('numbers.csv')
for i in range(len(df_numbers['words'])):
    if '-' in df_numbers['words'][i]:
        df_numbers['words'][i]=df_numbers['words'][i].replace('-',' ')
    
numbers_list=[]
for numbers in df_numbers['words']:
    numbers_list.append(numbers)
    
#list of all the alphabets    
alphabet_list=[]
alphabet_list.extend(['alfa', 'alpha', 'bravo', 'charlie', 'delta', 'echo', 'foxtrot', 'golf', 'hotel', 'india', 'juliett', 'kilo', 'lima', 'mike', 'november', 'oscar', 'papa', 'quebec', 'romeo', 'sierra', 'tango', 'uniform', 'victor', 'whiskey', 'xray', 'yankee', 'zulu'])
alphabet_list.extend([x for x in "abcdefghijklmnopqrstuvwxyz"])

In [2]:
metadata_path = 'metadata.csv'
metadata_df = pd.read_csv(metadata_path)
metadata_df.drop(columns=metadata_df.columns[0], axis=1, inplace=True)
metadata_df.drop(columns=metadata_df.columns[0], axis=1, inplace=True)
metadata_df = metadata_df.sample(frac=1,random_state=0).reset_index(drop=True)

split1 = int(len(metadata_df) * 0.95) #95% training set #5% test set
df_train = metadata_df[:split1]
df_test = metadata_df[split1:]
print(f"Size of the training set: {len(df_train)}")
df_train.head()

Size of the training set: 22512


,wav_filename,transcript
0,resampled_audio_14440,air berlin eight four three nine
1,resampled_audio_25292,csa six three five runway three one clear to l...
2,resampled_audio_8178,hapag lloyd three five five one descend now to...
3,resampled_audio_4829,sunwing five two one two contact zurich one th...
4,resampled_audio_12516,eurotrans four eight zero


In [4]:
#list of all callsigns to iterate through
callsign_list=[
' aviva ' ,
' adria ' ,
' ascot ' ,
' air atlantic ' ,
' aero lloyd ' ,
' aeroflot ',
' air inter ',
' aerosvit ',
' aerovic ',
' air berlin ',
' air france ',
' air malta ',
' air ukraine ',
' airfrans ',
' airlink ',
' algerie ',
' alitalia ',
' american ',
' atlanta ',
' austrian ',
' belgian ',
' belgian airforce ',
' belstar ',
' berlin ',
' bizex ',
' black sea ',
' blacksea ',
' blue ',
' bline ',
' beline ',
' brit air ',
' britair ',
' britannia ',
' canada ',
' cathay ',
' calibra ',
' central charter ',
' clipper ',
' condor ',
' cotam ',
' constellation ',
' continental ',
' cross air ',
' csa ',
' czech force ',
# ' delta ',
' dynasty ',
' easy ',
' egyptair ',
' emirates ',
' europa ',
' eurotrans ',
' etiopian ',
' fedex ',
' finnair ',
' fox sierra india ',
' fox oscar kilo sierra india ',
' french air force ',
' french line ',
' georgia air ',
' german air force ',
' german cargo ',
' germanwings ',
' german wings ',
' germania ',
' golf juliett india ',
' gulf air ',
' hamburg air ',
' hansa ',
' hapag lloyd ',
' hotel tango romeo ',
' hotel bravo india tango romeo ',
' iberia ',
' israeli air ',
' india sierra alfa ',
' jat ',
' jet link ',
' jet set ',
' jetcom ',
' jetaviation ',
' job air ',
' jobair ',
' jobair netherlands ',
' k l m ',
' klm ',
' l t u ',
' lauda ',
' lufthansa ',
' lufthansa cargo ',
' leisure ',
' luxair ',
' london airtours ',
' malaysian ',
' martinair ',
' merair ',
' meridiana ',
' malev ',
' midland ',
' mission ',
' netherlands ',
' netherlands air ',
' netherlands air force ',
' norshuttle ',
' northshuttle ',
' northwest ',
' olympic ',
' omanair ',
' operajet ',
' oscar hotel india ',
' p s a ',
' portugal ',
' pakistan ',
' portugalia ',
' ryanair ',
' sabena ',
' saudia ',
' scandinavian ',
' sunturk ',
' shamrock ',
' skyfox ',
' skytravel ',
' skyshuttle ',
' sobelair ', 
' southernair ',
' southern air ',
' speedbird ',
' speedway ',
' sunexpress ',
' sky master ',
' singapore ',
' sunwing ',
' swedestar ',
' swiss ',
' swissair ',
' thai ',
' thompson ', 
' thomson ',
' timeair ',
' topswiss ',
' transavia ',
' transwede ',
' turkair ',
' tunair ',
' tunis air ',
' turkish ',
' twinstar ',
' u p s ',
' ups ',
' u s air ',
' ukraine international ',
' united ',
' vietnam ',
' virgin ',
' viva ',
' wizzair '
]

In [5]:
#rule based approach to label call signs
train_data=[]

for utterance in df_train['transcript']:  
    for callsign in callsign_list:    
        if callsign in ' '+utterance+' ':
            current_index=0
            utterance_splitted = utterance.split()            
            temp=[]
            i=0

            while i<len(utterance_splitted):

                
                    if utterance_splitted[i] == callsign.split()[0]:
                        if i!=0:
                            current_index+=1
                        start_index = current_index
                        current_index+=len(utterance_splitted[i])
                        
                        i+=1
                        for j in range(1,len(callsign.split())):
                            current_index+=len(callsign.split()[j])
                            current_index+=1
                            i+=1
                        try:    
                            while utterance_splitted[i] in numbers_list \
                            or utterance_splitted[i] in alphabet_list:

                                current_index+=len(utterance_splitted[i])
                                current_index+=1

                                i+=1

                                if utterance_splitted[i-1] in alphabet_list and utterance_splitted[i] not in alphabet_list:

                                    break
                        except:
                            pass
                        end_index=current_index
                        temp.extend([(start_index,end_index,'CALLSIGN')])
                        
                    else:
                        current_index+=len(utterance_splitted[i])
                        if i!=0:
                            current_index+=1
                        i+=1
                
            train_data.append((utterance,{'entities':temp}))    
 

In [8]:
with open('spacy_ner/train_data', "wb") as file_pi:
    pickle.dump(train_data,file_pi)
with open('spacy_ner/train_data', 'rb') as outputfile:
    train_data = pickle.load(outputfile) 
train_data

[('air berlin eight four three nine', {'entities': [(0, 32, 'CALLSIGN')]}),
 ('air berlin eight four three nine', {'entities': [(4, 32, 'CALLSIGN')]}),
 ('csa six three five runway three one clear to land wind one six zero degrees three knots',
  {'entities': [(0, 18, 'CALLSIGN')]}),
 ('hapag lloyd three five five one descend now to flight level two five zero',
  {'entities': [(0, 31, 'CALLSIGN')]}),
 ('sunwing five two one two contact zurich one three four decimal six',
  {'entities': [(0, 24, 'CALLSIGN')]}),
 ('eurotrans four eight zero', {'entities': [(0, 25, 'CALLSIGN')]}),
 ('aeroflot one four one turn right heading two two zero vec',
  {'entities': [(0, 21, 'CALLSIGN')]}),
 ('fox oscar kilo sierra india good morning radar contact',
  {'entities': [(0, 27, 'CALLSIGN')]}),
 ('central charter one five eight praha radar contact',
  {'entities': [(0, 30, 'CALLSIGN')]}),
 ('csa six three five turn left heading three four zero cleared ils approach runway three one report when establishe

## Training

In [7]:
import pickle
import random
import spacy
from tqdm import tqdm
from spacy.training import Example


In [ ]:
model = None
output_dir='spacy_ner/spacy_ner_trained'
n_iter=100

In [12]:
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")



Created blank 'en' model


In [13]:
if 'ner' not in nlp.pipe_names:
    nlp.add_pipe('ner')
    ner = nlp.get_pipe('ner')
else:
    ner = nlp.get_pipe('ner')

In [14]:
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [17]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        for text, annotations in tqdm(train_data):
            doc=nlp.make_doc(text)
            example=Example.from_dict(doc,annotations)
            nlp.update(
                [example],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)
        
nlp.to_disk(output_dir)
print("Saved model to", output_dir)    

 10%|███████▋                                                                     | 1499/15112 [01:23<12:37, 17.97it/s]


KeyboardInterrupt: 

## Test

In [10]:
output_dir='spacy_ner/spacy_ner_trained/'

In [11]:
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)


Loading from spacy_ner/spacy_ner_trained/


In [16]:
for text, _ in test_data:
    doc = nlp2(text)
    
    print('Entities:', [(ent.text, ent.label_) for ent in doc.ents])  
    print('Text:',text)
    print('-'*100)

Entities: [('swiss one two four b', 'CALLSIGN')]
Text: swiss one two four b turn left heading zero one zero
----------------------------------------------------------------------------------------------------
Entities: [('swissair nine three five nine', 'CALLSIGN')]
Text: swissair nine three five nine is identified
----------------------------------------------------------------------------------------------------
Entities: [('austrian seven one one z', 'CALLSIGN')]
Text: left heading three hundred austrian seven one one z
----------------------------------------------------------------------------------------------------
Entities: [('easy four six two zero', 'CALLSIGN')]
Text: praha easy four six two zero
----------------------------------------------------------------------------------------------------
Entities: [('csa seven three two two', 'CALLSIGN')]
Text: csa seven three two two line up runway one three
----------------------------------------------------------------------------

Entities: [('alitalia three six nine', 'CALLSIGN')]
Text: alitalia three six nine contact zurich one three four decimal six
----------------------------------------------------------------------------------------------------
Entities: [('aero lloyd five one seven', 'CALLSIGN')]
Text: aero lloyd five one seven contact now zurich on one three four decimal six good bye
----------------------------------------------------------------------------------------------------
Entities: [('speedbird one two four', 'CALLSIGN')]
Text: speedbird one two four call rhein on one two seven three seven good day
----------------------------------------------------------------------------------------------------
Entities: [('delta seventeen seventy six', 'CALLSIGN')]
Text: delta seventeen seventy six so long
----------------------------------------------------------------------------------------------------
Entities: [('csa three seven b', 'CALLSIGN')]
Text: csa three seven b descend flight level one one ze

Entities: [("delta echo charlie i'm", 'CALLSIGN')]
Text: ah delta echo charlie i'm not sure whether you got it you proceed to dinkelsbuhl
----------------------------------------------------------------------------------------------------
Entities: [('ryanair eight seven y m', 'CALLSIGN')]
Text: ryanair eight seven y m hello radar contact
----------------------------------------------------------------------------------------------------
Entities: [('skytravel one zero three', 'CALLSIGN')]
Text: clear take off ehm runway one three skytravel one zero three
----------------------------------------------------------------------------------------------------
Entities: [('csa seven two six', 'CALLSIGN')]
Text: csa seven two six radar radar contact ehm climb flight level nine zero
----------------------------------------------------------------------------------------------------
Entities: [('skytravel six seven zero', 'CALLSIGN')]
Text: skytravel six seven zero praha radar contact ehm climb

Entities: [('sabena eight three two', 'CALLSIGN')]
Text: sabena eight three two contact rhein one two seven decimal three seven
----------------------------------------------------------------------------------------------------
Entities: [('lufthansa four four one six', 'CALLSIGN')]
Text: lufthansa four four one six good morning ah radar contact maintain flight level two nine zero your destination
----------------------------------------------------------------------------------------------------
Entities: [('csa eight two eight one', 'CALLSIGN')]
Text: csa eight two eight one reduce speed two three zero or less reducing two three zero less see es
----------------------------------------------------------------------------------------------------
Entities: [('ryanair nine one seven three', 'CALLSIGN')]
Text: climbing flight level three two zero ryanair nine one seven three
----------------------------------------------------------------------------------------------------
Entities: [(

Entities: [('lufthansa two five three nine', 'CALLSIGN')]
Text: lufthansa two five three nine descend flight level two nine zero level by letko
----------------------------------------------------------------------------------------------------
Entities: [('l t u seven seven five six', 'CALLSIGN')]
Text: l t u seven seven five six ah geneva one two eight one five good bye
----------------------------------------------------------------------------------------------------
Entities: [('csa eight m two', 'CALLSIGN')]
Text: csa eight m two contact radar one two zero . two seven five
----------------------------------------------------------------------------------------------------
Entities: [('sunwing five two zero three', 'CALLSIGN')]
Text: sunwing five two zero three good morning maintain level three four zero trasadingen tango
----------------------------------------------------------------------------------------------------
Entities: [('lufthansa four three nine three', 'CALLSIGN')]


...

Entities: [('ukraine international seven zero one', 'CALLSIGN')]
Text: good morning ukraine seven zero one level three four zero ukraine international seven zero one praha hello radar contact direct to o k g
----------------------------------------------------------------------------------------------------
Entities: [('delta november alfa', 'CALLSIGN')]
Text: delta november alfa turn left to fly heading one six five
----------------------------------------------------------------------------------------------------
Entities: [('lufthansa nine e l', 'CALLSIGN')]
Text: praha departure lufthansa nine e l climbing five thousand passing three thousand six
----------------------------------------------------------------------------------------------------
Entities: [('delta', 'CALLSIGN')]
Text: delta fox sierra climb to flight level three hundred
----------------------------------------------------------------------------------------------------
Entities: [('alitalia one three five', 'CALLS

Entities: [('alitalia three zero zero', 'CALLSIGN')]
Text: alitalia three zero zero bonjour i'll call you back
----------------------------------------------------------------------------------------------------
Entities: [('lufthansa five m e', 'CALLSIGN')]
Text: lufthansa five m e heading three three zero clear cleared ils approach three one
----------------------------------------------------------------------------------------------------
Entities: [('alitalia one one nine', 'CALLSIGN')]
Text: alitalia one one nine continue heading until advised
----------------------------------------------------------------------------------------------------
Entities: [('csa three c t', 'CALLSIGN')]
Text: csa three c t tower roger
----------------------------------------------------------------------------------------------------
Entities: [('adria four zero', 'CALLSIGN')]
Text: adria four zero vacate via g and contact ground one two one . nine g and one two one deci
----------------------------

Entities: [('aviva nine zero eight one', 'CALLSIGN')]
Text: aviva nine zero eight one turn right heading of zero nine zero
----------------------------------------------------------------------------------------------------
Entities: [('klm one eight three nine', 'CALLSIGN')]
Text: klm one eight three nine praha hello radar contact descend flight level two nine zero abrax at level
----------------------------------------------------------------------------------------------------
Entities: [('germanwings', 'CALLSIGN')]
Text: germanwings seve
----------------------------------------------------------------------------------------------------
Entities: [('alitalia four eight five', 'CALLSIGN')]
Text: alitalia four eight five rhein radar identified
----------------------------------------------------------------------------------------------------
Entities: [('easy three zero six five', 'CALLSIGN')]
Text: easy three zero six five hello radar radar contact no speed restrictions descend fli

Entities: [('lufthansa four six five two', 'CALLSIGN')]
Text: lufthansa four six five two contact geneva one three three one five bye bye
----------------------------------------------------------------------------------------------------
Entities: [('lufthansa', 'CALLSIGN')]
Text: lufthansa triple five zero own navigation to st prex
----------------------------------------------------------------------------------------------------
Entities: [('lufthansa six e u', 'CALLSIGN'), ('k t c', 'CALLSIGN')]
Text: lufthansa six e u proceed to k t c direct k t c lufthan
----------------------------------------------------------------------------------------------------
Entities: [('air france two two five', 'CALLSIGN')]
Text: air france two two five contact rhein one three six . four zero five good day
----------------------------------------------------------------------------------------------------
Entities: [('easy three eight p g', 'CALLSIGN')]
Text: easy three eight p g now ehm you are re

Entities: [('thai nine three one', 'CALLSIGN')]
Text: go ahead thai nine three one proceed direct sopav squawk one four seven four
----------------------------------------------------------------------------------------------------
Entities: [('turkish one seven six eight', 'CALLSIGN')]
Text: fl three three zero initially turkish one seven six eight
----------------------------------------------------------------------------------------------------
Entities: [('csa nine three zero', 'CALLSIGN')]
Text: praha radar csa nine three zero reaching level two eight zero
----------------------------------------------------------------------------------------------------
Entities: [('alitalia one one seven', 'CALLSIGN')]
Text: alitalia one one seven good morning radar contact
----------------------------------------------------------------------------------------------------
Entities: [('sabena seven eight one six', 'CALLSIGN')]
Text: sabena seven eight one six ah good morning radar contact
----

In [61]:
text='slow to one seventy united three five two heavy'
doc = nlp2(text)
print('Text:',text)
print('Entities:', [(ent.text, ent.label_) for ent in doc.ents])  

Text: slow to one seventy russia three five two heavy
Entities: []
